In [2]:
import math, time, random
from typing import List

class MemoryItem:
    def __init__(self, kind:str, content:str, score:float=1.0):
        self.kind = kind
        self.content = content
        self.score = score
        self.t = time.time()

class MemoryStore:
    def __init__(self, decay_half_life=1800):
        self.items: List[MemoryItem] = []
        self.decay_half_life = decay_half_life

    def _decay_factor(self, item:MemoryItem):
        dt = time.time() - item.t
        return 0.5 ** (dt / self.decay_half_life)

In [3]:
    def add(self, kind:str, content:str, score:float=1.0):
        self.items.append(MemoryItem(kind, content, score))

    def search(self, query:str, topk=3):
        scored = []
        for it in self.items:
            decay = self._decay_factor(it)
            sim = len(set(query.lower().split()) & set(it.content.lower().split()))
            final = (it.score * decay) + sim
            scored.append((final, it))
        scored.sort(key=lambda x: x[0], reverse=True)
        return [it for _, it in scored[:topk] if _ > 0]

    def cleanup(self, min_score=0.1):
        new = []
        for it in self.items:
            if it.score * self._decay_factor(it) > min_score:
                new.append(it)
        self.items = new

In [6]:
class Agent:
    def __init__(self, memory:MemoryStore, name="PersonalAgent"):
        self.memory = memory
        self.name = name

    def _llm_sim(self, prompt:str, context:List[str]):
        base = "OK. "
        if any("prefers short" in c for c in context):
            base = ""
        reply = base + f"I considered {len(context)} past notes. "
        if "summarize" in prompt.lower():
            return reply + "Summary: " + " | ".join(context[:2])
        if "recommend" in prompt.lower():
            if any("cybersecurity" in c for c in context):
                return reply + "Recommended: write more cybersecurity articles."
            if any("rag" in c for c in context):
                return reply + "Recommended: build an agentic RAG demo next."
            return reply + "Recommended: continue with your last topic."
        return reply + "Here's my response to: " + prompt

    def perceive(self, user_input:str):
        ui = user_input.lower()
        if "i like" in ui or "i prefer" in ui:
            self.memory.add("preference", user_input, 1.5)
        if "topic:" in ui:
            self.memory.add("topic", user_input, 1.2)
        if "project" in ui:
            self.memory.add("project", user_input, 1.0)
    def act(self, user_input:str):
        mems = self.memory.search(user_input, topk=4)
        ctx = [m.content for m in mems]
        answer = self._llm_sim(user_input, ctx)
        self.memory.add("dialog", f"user said: {user_input}", 0.6)
        self.memory.cleanup()
        return answer, ctx

In [7]:
def evaluate_personalisation(agent:Agent):
    agent.memory.add("preference", "User likes cybersecurity articles", 1.6)
    q = "Recommend what to write next"
    ans_personal, _ = agent.act(q)
    empty_mem = MemoryStore()
    cold_agent = Agent(empty_mem)
    ans_cold, _ = cold_agent.act(q)
    gain = len(ans_personal) - len(ans_cold)
    return ans_personal, ans_cold, gain

In [9]:
mem = MemoryStore(decay_half_life=60)
agent = Agent(mem)

print("=== Demo: teaching the agent about yourself ===")
inputs = [
    "I prefer short answers.",
    "I like writing about RAG and agentic AI.",
    "Topic: cybersecurity, phishing, APTs.",
    "My current project is to build an agentic RAG Q&A system."
]
for inp in inputs:
    agent.perceive(inp)

print("\n=== Now ask the agent something ===")
user_q = "Recommend what to write next in my blog"
ans, ctx = agent.act(user_q)
print("USER:", user_q)
print("AGENT:", ans)
print("USED MEMORY:", ctx)

print("\n=== Evaluate personalisation benefit ===")
p, c, g = evaluate_personalisation(agent)
print("With memory :", p)
print("Cold start  :", c)
print("Personalisation gain (chars):", g)

print("\n=== Current memory snapshot ===")
for it in agent.memory.items:
    print(f"- {it.kind} | {it.content[:60]}... | score~{round(it.score,2)}")

=== Demo: teaching the agent about yourself ===

=== Now ask the agent something ===
USER: Recommend what to write next in my blog
AGENT: OK. I considered 4 past notes. Recommended: write more cybersecurity articles.
USED MEMORY: ['My current project is to build an agentic RAG Q&A system.', 'I prefer short answers.', 'I like writing about RAG and agentic AI.', 'Topic: cybersecurity, phishing, APTs.']

=== Evaluate personalisation benefit ===
With memory : OK. I considered 4 past notes. Recommended: write more cybersecurity articles.
Cold start  : OK. I considered 0 past notes. Recommended: continue with your last topic.
Personalisation gain (chars): 4

=== Current memory snapshot ===
- preference | I prefer short answers.... | score~1.5
- preference | I like writing about RAG and agentic AI.... | score~1.5
- topic | Topic: cybersecurity, phishing, APTs.... | score~1.2
- project | My current project is to build an agentic RAG Q&A system.... | score~1.0
- dialog | user said: Recommend wh